#### フォルダ内の.pptx .pdfを.csvに出力

In [ ]:
import docx
import docx2txt
import openpyxl
import pptx
from pptx.exc import PackageNotFoundError
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfpage import PDFTextExtractionNotAllowed
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.converter import PDFPageAggregator
from pdfminer.layout import LTTextBoxHorizontal
from pdfminer.layout import (
    LAParams,
    LTContainer,
    LTTextLine,
)
import glob
import re
import os
import pandas as pd
import numpy as np
from openpyxl import load_workbook
import csv


DIR = r'C:\tmp\**'
files = glob.glob(DIR, recursive=True)
#print("recursive", files)

ix1 = 0
for file in files:
    
    # .pptx ファイル
    if  file.endswith('.pptx') :
        # 抽出したテキストデータを格納する空リスト
        extracted_texts = []

        # テキストデータが抽出されたpptxファイルのパスを格納する空リスト
        pathes = []

        # テキストデータが抽出されたスライドのページを格納する空リスト
        slide_num = []
        
        print(file)
        prs = pptx.Presentation(file)
        print(extracted_texts)
        
        # スライドごとにテキストデータを抽出する
        for i, sld in enumerate(prs.slides, start=1):
            for shape in sld.shapes:
                if shape.has_text_frame: # shapeに含まれるテキストデータを抽出
                    pathes.append(file)
                    slide_num.append(i)
                    extracted_texts.append(shape.text)

                if shape.has_table: # tableに含まれるテキストデータを抽出
                    for cell in shape.table.iter_cells():
                        pathes.append(file)
                        slide_num.append(i)
                        extracted_texts.append(cell.text)

        #print(extracted_texts)
        fn = file + ".csv"
        # テキストファイル書き込み
        with open(fn, 'w',encoding='utf-8_sig') as f:
            for i in extracted_texts:
                f.write(str(i) + "\n")
                       
        ix1 = ix1 + 1
        
    # .pdf ファイル
    if  file.endswith('.pdf') :
        # 抽出したテキストデータを格納する空リスト
        extracted_texts = []

        fp = open(file, 'rb')
        parser = PDFParser(fp)
        document = PDFDocument(parser)

        if not document.is_extractable:
            raise #PDFTextExtractionNotAllowed
        rsrcmgr = PDFResourceManager()
        laparams = LAParams()
        device = PDFPageAggregator(rsrcmgr, laparams=laparams)

        interpreter = PDFPageInterpreter(rsrcmgr, device)
        #pages = list(document.get_pages())

        for page in PDFPage.create_pages(document):
            interpreter.process_page(page)
            layout = device.get_result()
            for l in layout:
                if isinstance(l, LTTextBoxHorizontal):
                    text = l.get_text()
                    extracted_texts.append(text)


        #print(extracted_texts)
        fn = file + ".csv"
        # テキストファイル書き込み
        with open(fn, 'w',encoding='utf-8_sig') as f:
            for i in extracted_texts:
                f.write(str(i) + "\n")
                
        ix1 = ix1 + 1
        
    
print(ix1)
